In [ ]:
!pip install simpletransformers


     |████████████████████████████████| 215kB 11.8MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 194kB 48.1MB/s 
     |████████████████████████████████| 2.2MB 53.9MB/s 
     |████████████████████████████████| 3.3MB 50.7MB/s 
     |████████████████████████████████| 8.2MB 42.3MB/s 
     |████████████████████████████████| 122kB 54.9MB/s 
     |████████████████████████████████| 81kB 3.2MB/s 
     |████████████████████████████████| 2.1MB 48.6MB/s 
     |████████████████████████████████| 1.2MB 44.8MB/s 
     |████████████████████████████████| 245kB 52.5MB/s 
     |████████████████████████████████| 112kB 59.1MB/s 
     |████████████████████████████████| 870kB 48.3MB/s 
     |████████████████████████████████| 4.6MB 44.2MB/s 
     |████████████████████████████████| 163kB 54.4MB/s 
     |████████████████████████████████| 112kB 49.7MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 133kB 62.2MB/s

In [ ]:
import pandas as pd
import numpy as np

import logging
import wandb

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import classification_report, confusion_matrix

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')

from nltk.stem import LancasterStemmer, WordNetLemmatizer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#Creating Logger
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

### Import Data

In [ ]:
#Creating the dataframes
kiva_train = pd.read_csv('https://raw.githubusercontent.com/DDave94/Loan-Default-Prediction/main/kiva_train.csv')
kiva_test = pd.read_csv('https://raw.githubusercontent.com/DDave94/Loan-Default-Prediction/main/kiva_test.csv')

#Formatting kaggle dataframe to run in the predict function
kaggle_test = kiva_test['en_clean']
kaggle_test = kaggle_test.to_list()

kiva_train.head(10)

,loan_id,en_clean,defaulted
0,7779,She opened a colmado out of the side of her ho...,0
1,2777,(First Loan): Joffre continues to run his loc...,1
2,6007,"Dina Santana is the mother of two children, Ju...",0
3,76,"Rosemary is 50 years old, single, and has 6 ch...",1
4,4217,"Segundo has a shop where he sells animal feed,...",0
5,5077,"I am a single parent, mother of 4. I sell groc...",0
6,6033,Mariana Jose Serda owns a general store in Mag...,0
7,843,Mary is a mother of four children. One of the ...,1
8,4357,Wilson is seeking his second loan with Kiva an...,0
9,7007,Clara lives with her two children (ages: 20 an...,0


### Preprocessing

In [ ]:
pip install unidecode textstat

     |████████████████████████████████| 245kB 14.4MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 1.9MB 14.4MB/s 


In [ ]:
# import unicodedata
# import unidecode
# import re
# stop_words = set(stopwords.words('english') + stopwords.words('spanish'))
# # Lemmatization is better than stemming
# lemmer = WordNetLemmatizer()
# #lemmer = LancasterStemmer()

# def preprocess(x):
#     # Lower case
#     x = x.lower()
    
#     # Remove punctuation
#     x = re.sub(r'[^\w\s]', '', x)

#     # Remove numbers
#     # x = re.sub(r'\d+', '', x)
    
#     # Remove non-unicode
#     x = unidecode.unidecode(x)
        
#     # Remove stopwords and lemmatize
#     # x = [lemmer.lemmatize(w) for w in x.split() if w not in stop_words]
#     x = [lemmer.lemmatize(w) for w in x.split()]
#     #x = [lemmer.stem(w) for w in x.split()]
#     return ' '.join(x) 

# kiva_train['en_clean'] = kiva_train['en_clean'].apply(preprocess)

### Looking at Training Data after preprocessing

In [ ]:
kiva_train.head(10)

,loan_id,en_clean,defaulted
0,7779,She opened a colmado out of the side of her ho...,0
1,2777,(First Loan): Joffre continues to run his loc...,1
2,6007,"Dina Santana is the mother of two children, Ju...",0
3,76,"Rosemary is 50 years old, single, and has 6 ch...",1
4,4217,"Segundo has a shop where he sells animal feed,...",0
5,5077,"I am a single parent, mother of 4. I sell groc...",0
6,6033,Mariana Jose Serda owns a general store in Mag...,0
7,843,Mary is a mother of four children. One of the ...,1
8,4357,Wilson is seeking his second loan with Kiva an...,0
9,7007,Clara lives with her two children (ages: 20 an...,0


### Train / Val Data

In [ ]:
# Creating Training Data
X = kiva_train['en_clean']
y = kiva_train['defaulted']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_df = pd.concat([X_train,y_train], axis = 1)
eval_df = pd.concat([X_val,y_val],axis = 1)

### DL Model

In [ ]:
# Create arguements
model_args = ClassificationArgs()
model_args.eval_batch_size = 8
model_args.evaluate_during_training = False
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.learning_rate = 4e-5
model_args.manual_seed = 20
model_args.max_seq_length = 300
model_args.no_cache = False # True for 0.90
model_args.num_train_epochs = 8
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.train_batch_size = 32 # 32 for 0.90
#model_args.gradient_accumulation_steps = 2
model_args.output_dir = "/content/default_output"
model_args.best_model_dir = "/content/default_output/best_model"
model_args.wandb_project = "RTE - Hyperparameter Optimization"
model_args.wandb_kwargs = {"name": "default"}

#trying lemmitization with stopwords, lower case, an removing punctuation (0.87)
# no multiprocessing and no preprocessing (0.90)
# using eval batch size 16 (0.90)


# Create a Classification Model
model = ClassificationModel("bert", "bert-base-cased", num_labels = 2 , weight= [0.5, 2], args=model_args)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/4910 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_300_2_2


Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for training.


...

Running Evaluation:   0%|          | 0/154 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


Training loss,0.01332
lr,0.0
global_step,1200
_runtime,1177
_timestamp,1618200998
_step,23


Training loss,█▆▆▄▅▂▂▂▃▅▁▆▆▂▁▁▁▁▁▁▂▁▁▁
lr,▆██▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▁▁
global_step,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██
_runtime,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██
_timestamp,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██
_step,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██


Exception ignored in: <finalize object at 0x7fb5e0d15ed0; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/usr/lib/python3.7/tempfile.py", line 936, in _cleanup
    _rmtree(name)
  File "/usr/lib/python3.7/shutil.py", line 485, in rmtree
    onerror(os.lstat, path, sys.exc_info())
  File "/usr/lib/python3.7/shutil.py", line 483, in rmtree
    orig_st = os.lstat(path)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpuvcoyres'
Exception ignored in: <finalize object at 0x7fb2d1b46b50; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/usr/lib/python3.7/tempfile.py", line 936, in _cleanup
    _rmtree(name)
  File "/usr/lib/python3.7/shutil.py", line 485, in rmtree
    onerror(os.lstat, path, sys.exc_info())
  File "/usr/lib/python3.7/shutil.p

In [ ]:
preds, outputs = model.predict(X_val.to_list())


INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1228 [00:00<?, ?it/s]

  0%|          | 0/154 [00:00<?, ?it/s]

In [ ]:
print(classification_report(y_val, preds))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90       629
           1       0.91      0.89      0.90       599

    accuracy                           0.90      1228
   macro avg       0.90      0.90      0.90      1228
weighted avg       0.90      0.90      0.90      1228



In [ ]:
print(confusion_matrix(y_val, preds))

[[546  83]
 [ 76 523]]
